In [ ]:
from tensorflow import keras
import numpy as np

In [ ]:
import random
nums = [0]

In [ ]:
def make_time_series(iter):
    nums= [0]
    
    for _ in range(iter):
        for i in range(10):
            nums.append(nums[-1]+np.random.normal(5,1))
        for i in range(10):
            nums.append(nums[-1]-np.random.normal(5,1))
    return np.array(nums)[1:]

In [ ]:
num_array = make_time_series(100)
num_array.shape

(2000,)

In [ ]:
num_array = num_array.reshape((-1, 10))
num_array = num_array[:,:, np.newaxis]

num_array.shape

(200, 10, 1)

In [ ]:
enc_input = num_array[:num_array.shape[0]-1]
dec_target, dec_input = num_array[1:],num_array[1:]
enc_input.shape, dec_input.shape

((199, 10, 1), (199, 10, 1))

In [ ]:
encoder_inputs = keras.layers.Input(shape=(10, 1), name="encoder_input")
encoder_lstm = keras.layers.LSTM(units=64, 
                                 return_state=True,
                                 name="encoder_lstm")

encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

encoder_states = [state_h, state_c]

In [ ]:
decoder_inputs = keras.layers.Input(shape=(None,1),
                                    name="decoder_input")

decoder_lstm = keras.layers.LSTM(units=64, 
                                 return_sequences=True, 
                                 return_state=True,                             
                                 name="decoder_lstm")
  
decoder_outputs, decoder_state_h, decoder_state_c = decoder_lstm(decoder_inputs, 
                                                                 initial_state=encoder_states)

decoder_dense = keras.layers.TimeDistributed(keras.layers.Dense(1))
decoder_outputs = decoder_dense(decoder_outputs)



In [ ]:
model = keras.Model([encoder_inputs,decoder_inputs], decoder_outputs)
model.compile(optimizer="adam", loss="mse")

In [ ]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input (InputLayer)     [(None, 10, 1)]      0           []                               
                                                                                                  
 decoder_input (InputLayer)     [(None, None, 1)]    0           []                               
                                                                                                  
 encoder_lstm (LSTM)            [(None, 64),         16896       ['encoder_input[0][0]']          
                                 (None, 64),                                                      
                                 (None, 64)]                                                      
                                                                                            

return_sequence는 return_sequences=True로 설정하면 RNN 층의 출력이 시퀀스로 반환되므로 다음 RNN 층의 입력으로 사용됩니다. 이렇게 하면 이전 타임스텝에서 계산된 출력을 현재 타임스텝의 입력으로 사용할 수 있으므로 모델이 더욱 정확하게 예측을 수행할 수 있습니다. 이를 교사 강요(teacher forcing)라고 합니다.

https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=chunjein&logNo=221589624838

만약 return_sequences=False로 설정하면 RNN 층의 출력이 마지막 타임스텝의 출력만 반환되므로, 다음 층의 입력으로는 마지막 타임스텝의 출력만 사용됩니다. 이는 일반적으로 sequence-to-sequence 모델에서 디코더 층에서만 사용됩니다.

최종 출력 층인 dense층을 time_distributed 층으로 감싸주는 이유는 매 타입스텝마다 발생하는 손실을 반영하기 위함이다. 

In [ ]:
model.fit(
    x=[enc_input, dec_input],
    y=dec_target,
    epochs=10
    )

Epoch 1/10
7/7 [==============================] - 5s 13ms/step - loss: 401.4518
Epoch 2/10
7/7 [==============================] - 0s 12ms/step - loss: 381.3730
Epoch 3/10
7/7 [==============================] - 0s 12ms/step - loss: 361.8965
Epoch 4/10
7/7 [==============================] - 0s 14ms/step - loss: 342.2415
Epoch 5/10
7/7 [==============================] - 0s 12ms/step - loss: 321.8994
Epoch 6/10
7/7 [==============================] - 0s 12ms/step - loss: 301.5331
Epoch 7/10
7/7 [==============================] - 0s 13ms/step - loss: 284.0751
Epoch 8/10
7/7 [==============================] - 0s 13ms/step - loss: 268.6053
Epoch 9/10
7/7 [==============================] - 0s 16ms/step - loss: 254.9967
Epoch 10/10
7/7 [==============================] - 0s 13ms/step - loss: 242.6561


In [ ]:
pred = model.predict([enc_input,dec_input])

7/7 [==============================] - 1s 8ms/step


In [ ]:
pred = model.predict([enc_input[0][np.newaxis,:,:], dec_input[0][np.newaxis,:,:]])

1/1 [==============================] - 0s 23ms/step


In [ ]:
input_data = tf.ones(shape=(1, 10, 1))
_, state_h_value, _ = encoder_lstm(input_data)
state_h_value_numpy = state_h_value.numpy()
print(state_h_value_numpy)

[[0.33372492]]


In [ ]:
len(model.get_weights())

8

In [ ]:
for i in range(len(model.get_weights())):
    print(model.get_weights()[i].shape)

(1, 4)
(1, 4)
(4,)
(1, 4)
(1, 4)
(4,)
(1, 1)
(1,)


위 가중치는 LSTM 셀이 하나일때 나타나는 가중치 개수이다. 

각각
- encoder_lstm의 입력 가중치
- encoder_lstm의 숨겨진 상태 가중치
- encoder_lstm의 편향 가중치
- decoder_lstm의 입력 가중치
- decoder_lstm의 숨겨진 상태 가중치
- decoder_lstm의 편향 가중치
- decoder_dense의 가중치
- decoder_dense의 편향 가중치
이다. 

각각의 RNN셀들은 각 타임스텝에서 다음 타임스텝으로 넘어갈 때 완전히 연결된다. LSRM은 내부적으로 4개의 가중치가 있기 때문에 4라는 수치를 볼 수 있다.

Transformer 공부를 하다가 RNN의 동작 원리가 헷갈려서 다시 공부했다. 

내가 만든 모델은 seq2seq모델이고 디코더에 입력을 넣어서 교사강요 방식으로 학습이 진행된다. 

그렇기 때문에 학습시와 예측시의 모델 작동이 다르게 이루어진다. (이부분은 아직 모르겠음)

굳이 encoder-decoder로 나누지 않고, many to one 방식으로 만들수도 있다. 그러면 바로 다음 타임스텝값을 예측하는 모델이 만들어 지는 것이다. 

또한 이번 실습을 통해서 rnn 셀을 연결과 가중치에 대해서 간단하게 알 수 있었다.


하지만 궁금한 점이 하나가 있는데, LSTM셀이 잘 작동하고 어떻게 사용하는지 간단하게 알았지만 정확히 어떻게 내부적으로 동작하는지는 잘 모르겠다. 

4개의 게이트가 어떻게 작동하는 건지는 잘모르겠다는 말이다. 이부분은 그냥 내가 공부를 덜 해서 그런건지 아니면 그냥 모델이 블랙박스로 이루어졌기 떄문인지 잘 모르겠다. 

내가 왜 내부적인 작동을 알고 싶냐면 내부적인 작동을 알고 시계열 데이터에 학습이 어떻게 진행되는 지 궁금하기 떄문이다. 만약 큰 급등이 있고 차차 하락하는 시계열 데이터라면 큰 급등을 장기적으로 기억하는 것인지... 이런 부분이 궁금하다. 암튼 이부분은 천천히 고민해봐야할듯하다. 